In [ ]:
import os
import pickle

import numpy as np
import h5py
import matplotlib.pyplot as plt
import torch

import PIL
from skimage import io

In [ ]:
import openslide

In [ ]:
try:
    import multiresolutionimageinterface as mir
except ImportError as e:
    print(e)

In [ ]:
from utils.file_utils import print_h5_file_info, check_h5_files_are_equal

In [ ]:
camelyon16_data_path = 'data/camelyon16-test001'
camelyon16_patch_size = 256
subject_id = 'test_001'

# parameters used by authors for CAMELYON16 dataset
mask_downsapmle_level = 5
mask_downsample_factor = 2 ** mask_downsapmle_level

# from the original CAMELYON16 dataset
sample_slide_path = f'{camelyon16_data_path}/slides/{subject_id}.tif'
sample_xml_annotation_path = f'{camelyon16_data_path}/lesion_annotations/{subject_id}.xml'

# computed by the authors using the xml file and slide dimentions using multiresolutionimageinterface (example at the very end of the notebook)
sample_reference_mask_path = f'{camelyon16_data_path}/reference_masks/{subject_id}.png'

# patches shared by authours using the CLAM pipeline
sample_h5_patches_fp = f"{camelyon16_data_path}/patches_fp_shared_by_authors/patches/{subject_id}.h5"

# ground truth patch indexes computed by authors for the patches created by CLAM pipeline using patch coordinates and lesion annotations xml file
sample_pickle_gt_path = f'{camelyon16_data_path}/gt_patches_indexes/{subject_id}.pkl'

# features computed by authors after creating patches with CLAM pipeline
sample_h5_features_file_path = f'{camelyon16_data_path}/features_shared_by_authors/h5_files/{subject_id}.h5'
sample_pt_features_file_path = f'{camelyon16_data_path}/features_shared_by_authors/pt_files/{subject_id}.pt'
if not os.path.exists(sample_pt_features_file_path):
    with h5py.File(sample_h5_features_file_path, "r") as f:
        feats_pt_from_hdf5 = torch.from_numpy(f['features'][:])
        torch.save(feats_pt_from_hdf5, sample_pt_features_file_path)

# can be computed using 'coords' (either patches or features file) from the h5 files shared by the authors
tissue_mask_downsampled_path = f"{camelyon16_data_path}/tissue_masks_created_from_h5_coords/{subject_id}.png"

# computed by GeorgeBatch from the xml file using ASAP multiresolutionimageinterface (mir)
sample_from_xml_reference_mask_mir_path = f'{camelyon16_data_path}/masks_created_from_xml/{subject_id}_ASAP_mir.tif'
sample_from_xml_reference_mask_openslide_path = f'{camelyon16_data_path}/masks_created_from_xml/{subject_id}_ASAP_openslide.tif'

In [ ]:
# can be changed - just used as an example of a region where the lesion is present
example_region_downsample_level = 4
example_region_xy = (14000, 10000)
example_region_size = (1000, 1000)

## Openslide

In [ ]:
openslide_slide = openslide.open_slide(sample_slide_path)

for i in range(openslide_slide.level_count):
    print(f'Level {i}')
    print(f'\t Dimensions (width, height): {openslide_slide.level_dimensions[i]}')
    print(f'\t Downsample: {openslide_slide.level_downsamples[i]}')

plt.imshow(openslide_slide.get_thumbnail((786, 786)))
plt.show()

In [ ]:
downsample_factor = openslide_slide.level_downsamples[example_region_downsample_level]
print("downsample_level :", example_region_downsample_level)
print("downsample_factor:", downsample_factor)
assert downsample_factor == 2 ** example_region_downsample_level

image_patch = openslide_slide.read_region(
    example_region_xy,
    example_region_downsample_level,
    example_region_size
)
plt.imshow(image_patch)
image_patch.size

In [ ]:
for key, value in openslide_slide.properties.items():
    print(key, value)

## Ground Truth Pickle files

In [ ]:
with open(sample_pickle_gt_path, 'rb') as f:
    gt_patches_indexes = pickle.load(f)

In [ ]:
type(gt_patches_indexes), len(gt_patches_indexes)

In [ ]:
gt_patches_indexes_np = np.array(gt_patches_indexes)

In [ ]:
gt_patches_indexes_np[:10], gt_patches_indexes_np[-10:]

In [ ]:
min(gt_patches_indexes_np), max(gt_patches_indexes_np), max(gt_patches_indexes_np) - min(gt_patches_indexes_np)

## Tumor (Reference) Mask from the original dataset

In [ ]:
print(sample_reference_mask_path)
sample_reference_mask = PIL.Image.open(sample_reference_mask_path)
print(sample_reference_mask.size) # (width, height)

plt.imshow(sample_reference_mask)
plt.show()

In [ ]:
# (total slide width, total slide hieght) / (total mask width, total mask height)
np.array(openslide_slide.dimensions) / np.array(sample_reference_mask.size)

In [ ]:
sample_reference_mask_np = np.array(sample_reference_mask)
print("shape:", sample_reference_mask_np.shape)  # (height, width, channels) <=> (row, col, colour)
print("unique:", np.unique(sample_reference_mask_np))
print("range:", sample_reference_mask_np.min(axis=(0, 1)), sample_reference_mask_np.max(axis=(0, 1)))

sample_reference_mask_np_larger_side = max(sample_reference_mask_np.shape[:2])
print("larger side:", sample_reference_mask_np_larger_side)

In [ ]:
sample_reference_mask_np_binary = (
    sample_reference_mask_np.mean(axis=2, keepdims=True) / 255).astype(int)

print("shape:", sample_reference_mask_np_binary.shape)
print("unique:", np.unique(sample_reference_mask_np_binary))
print("range:", sample_reference_mask_np_binary.min(), sample_reference_mask_np_binary.max())

In [ ]:
(sample_reference_mask_np == [255, 255, 255]).sum() / (mask_downsample_factor ** 2)

In [ ]:
slide_thumbnail_np = np.array(
    openslide_slide.get_thumbnail((sample_reference_mask_np_larger_side, sample_reference_mask_np_larger_side))
)
slide_thumbnail_np.shape

In [ ]:
plt.imshow(slide_thumbnail_np * sample_reference_mask_np_binary)
plt.show()

In [ ]:
alpha = 0.3
sample_reference_mask_np_transparency = np.clip(
    sample_reference_mask_np_binary,
    alpha, 1
)

plt.imshow((slide_thumbnail_np * sample_reference_mask_np_transparency).astype(int))
plt.show()

## patches_fp HDF5 file with 'coords' and metadata - shared by the authors

In [ ]:
# not using print_h5_file_info because we need to get coords_wo_features_np

print_h5_file_info(sample_h5_patches_fp)

with h5py.File(sample_h5_patches_fp, 'r') as f:
    coords_wo_features_np = f['coords'][:]

## features HDF5 file with 'coords' and 'features' shared by the authors

In [ ]:
print_h5_file_info(sample_h5_features_file_path)

with h5py.File(sample_h5_features_file_path, 'r') as f:
    tissue_coords_np = f['coords'][:]

print(tissue_coords_np.shape)
print(tissue_coords_np.min(axis=0), tissue_coords_np.max(axis=0))

In [ ]:
# check that the coordinates from the file with features and without match
assert (coords_wo_features_np == tissue_coords_np).all()

In [ ]:
print(sample_slide_path)
openslide_slide = openslide.open_slide(sample_slide_path)
openslide_slide.dimensions

In [ ]:
# check if the coordinates are multiples of 64 and 128
#   of 64, but not of 128 
(tissue_coords_np % 64 == 0).all(), (tissue_coords_np % 128 == 0).all()

In [ ]:
tissue_coords_np[gt_patches_indexes_np].shape

### Make a reference mask using shared 'coords' and gt_patch_indexes pickle

In [ ]:
# make a mask from tissue_coords_np / mask_downsample_factor taking the indexes from gt_patches_indexes_np to be 1 and all the other to be 0
# camelyon16 patch size was 256 in MS-CLAM

assert camelyon16_patch_size % mask_downsample_factor == 0
downsampled_patch_size = camelyon16_patch_size // mask_downsample_factor
print(downsampled_patch_size)

tissue_coords_np_downsampled = (tissue_coords_np / mask_downsample_factor).astype(int)

print(tissue_coords_np_downsampled.min(axis=0), tissue_coords_np_downsampled.max(axis=0))
print()


np.array(openslide_slide.dimensions) // mask_downsample_factor == np.array(sample_reference_mask_np.shape)[[1, 0]]
# make a 0-mask
tumor_mask_downsampled = np.zeros(
    np.array(openslide_slide.dimensions) // mask_downsample_factor,
    # tissue_coords_np_downsampled64.max(axis=0) + 1,
    dtype=int)
# set the indexes from gt_patches_indexes_np to 1
# plt.imshow(tumor_mask_downsampled,)


x_coords, y_coords = tissue_coords_np_downsampled[gt_patches_indexes_np].T
for x, y in zip(x_coords, y_coords):
    tumor_mask_downsampled[x:x+downsampled_patch_size,
                       y:y+downsampled_patch_size] = 1

print(tumor_mask_downsampled.min(), tumor_mask_downsampled.max())


tumor_mask_downsampled_transposed = tumor_mask_downsampled.T

# check that the masks match
assert (tumor_mask_downsampled_transposed ==
        sample_reference_mask_np_binary[:, :, 0]).all()

# bw = ListedColormap(['black', 'white'])
# plt.imshow(tumor_mask_downsampled, cmap=bw)
plt.imshow(tumor_mask_downsampled_transposed, cmap='gray')
plt.show()

In [ ]:
plt.imshow(
    PIL.Image.fromarray(
        (tumor_mask_downsampled_transposed * 255).astype(np.uint8)
        ) #.save('mask_downsampled_pil.png')
    , cmap='gray'
)
plt.show()

In [ ]:
sample_reference_mask_np_binary.shape, tumor_mask_downsampled_transposed.shape

In [ ]:
sample_reference_mask_np_binary[:, :, 0].max(
), tumor_mask_downsampled_transposed.max()

In [ ]:
masks_overlay = np.stack(
    # (red, green, blue) channels
    [
        sample_reference_mask_np_binary[:, :, 0],
        np.zeros_like(tumor_mask_downsampled_transposed),
        tumor_mask_downsampled_transposed,
    ],                  
    axis=-1
)
plt.imshow((PIL.Image.fromarray((masks_overlay * 255).astype(np.uint8))))
plt.show()

In [ ]:
# check that the masks match
assert (tumor_mask_downsampled_transposed == sample_reference_mask_np_binary[:, :,0]).all()

### Tissue Mask recovered using the 'coords' from the HDF5 file

In [ ]:
print(sample_h5_patches_fp)
print_h5_file_info(sample_h5_patches_fp)

In [ ]:
# make a mask from tissue_coords_np / 64 taking the indexes from gt_patches_indexes_np to be 1 and all the other to be 0
# use the minimum and maximum of the coordinates in both axis to make the mask

# camelyon16 patch size was 256 in MS-CLAM

assert camelyon16_patch_size % mask_downsample_factor == 0
downsampled_patch_size = camelyon16_patch_size // mask_downsample_factor
print(downsampled_patch_size)

tissue_coords_np_downsampled = (tissue_coords_np / mask_downsample_factor).astype(int)

print(tissue_coords_np_downsampled.min(axis=0), tissue_coords_np_downsampled.max(axis=0))
print()


np.array(openslide_slide.dimensions) // mask_downsample_factor == np.array(
    sample_reference_mask_np.shape)[[1, 0]]
# make a 0-mask
tissue_mask_downsampled = np.zeros(
    np.array(openslide_slide.dimensions) // mask_downsample_factor,
    # tissue_coords_np_downsampled64.max(axis=0) + 1,
    dtype=int)

x_coords, y_coords = tissue_coords_np_downsampled[:].T
for x, y in zip(x_coords, y_coords):
    tissue_mask_downsampled[x:x+downsampled_patch_size,
                     y:y+downsampled_patch_size] = 1

print(tissue_mask_downsampled.min(), tissue_mask_downsampled.max())


tissue_mask_downsampled_transposed = tissue_mask_downsampled.T

plt.imshow(tissue_mask_downsampled_transposed, cmap='gray')

os.makedirs(os.path.dirname(tissue_mask_downsampled_path), exist_ok=True)
plt.imsave(tissue_mask_downsampled_path, tissue_mask_downsampled_transposed, cmap='gray')

### Tissue Mask - Tumor mask overlay

In [ ]:
tissue_tumor_masks_overlay = np.stack(
    # (red, green, blue) channels
    [
        tissue_mask_downsampled_transposed,                 # red tissue   
        np.zeros_like(tissue_mask_downsampled_transposed),  # zero green
        tumor_mask_downsampled_transposed,                  # blue tumor
    ],
    axis=-1
)
plt.imshow(PIL.Image.fromarray((tissue_tumor_masks_overlay * 255).astype(np.uint8)))
plt.show()

## Check that the feature extraction was successful

Follow the instructions from the README, but use the single-slide example to speed up the process as described below.

To extract features, use [extract_features_fp.py](https://github.com/mahmoodlab/CLAM/blob/master/extract_features_fp.py) as shown below. You can speed up the process if you have a GPU with more memory or even multiple GPUs by
- increasing the `batch_size` parameter, e.g. to 1024
- setting `CUDA_VISIBLE_DEVICES=0,1,2,3` if you have 4 GPUs

```shell
# tmux new-session -s extract-camelyon16-test001-features
# tmux attach -t extract-camelyon16-test001-features
# conda activate msclam

# patches_fp has the coordinates shared by the authors of MS-CLAM
CUDA_VISIBLE_DEVICES=0 python extract_features_fp.py \
  --data_h5_dir ../MS-CLAM/data/camelyon16-test001/patches_fp_shared_by_authors/ \
  --data_slide_dir ../MS-CLAM/data/camelyon16-test001/slides \
  --csv_path ../MS-CLAM/data/camelyon16-test001/patches_fp_shared_by_authors/process_list.csv \
  --feat_dir ../MS-CLAM/data/camelyon16-test001/features/ \
  --batch_size 256 \
  --slide_ext .tif
```

In [ ]:
assert torch.allclose(
    torch.load(
        sample_pt_features_file_path, map_location='cpu'),
    torch.load(
        "data/camelyon16-test001/features/pt_files/test_001.pt", map_location='cpu'),
    atol=1e-7
)

In [ ]:
assert check_h5_files_are_equal(
    sample_h5_features_file_path,
    "data/camelyon16-test001/features/h5_files/test_001.h5",
    atol=1e-7
)

## Find Indexes all tissue patches within tumour polygons in XML files

This method should result in a similar set of indexes to the `gt_patches_indexes_np`

In [ ]:
import h5py
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon, Point


def parse_xml_to_polygons(xml_fp):
    tree = ET.parse(xml_fp)
    root = tree.getroot()

    polygons = []
    for annotation in root.find('Annotations').findall('Annotation'):
        coords = annotation.find('Coordinates')
        points = [(float(coord.attrib['X']), float(coord.attrib['Y'])) for coord in coords.findall('Coordinate')]
        polygon = Polygon(points)
        polygons.append(polygon)
    
    return polygons


def get_patch_centers(coords, patch_size):
    centers = []
    half_size = patch_size / 2
    for coord in coords:
        center_x = coord[0] + half_size
        center_y = coord[1] + half_size
        centers.append((center_x, center_y))
    return centers


def get_patches_within_polygons(patch_centers, polygons):
    indexes = []
    for i, center in enumerate(patch_centers):
        point = Point(center)
        if any(polygon.contains(point) for polygon in polygons):
            indexes.append(i)
    return indexes

# ---------------------------------------------------------------------------

# Parse the XML file to get the polygons
polygons = parse_xml_to_polygons(sample_xml_annotation_path)
print(f"{len(polygons)} Polygons:\n", polygons[:2], "...\n")

# Load the HDF5 file and get the coordinates of patches
with h5py.File(sample_h5_patches_fp, 'r') as f:
    coords_wo_features_np = f['coords'][:]
    patch_size = f['coords'].attrs['patch_size']

# Get the patch centers
patch_centers = get_patch_centers(coords_wo_features_np, patch_size)
print(f"{len(patch_centers)} Patch centers:\n", patch_centers[:5], "...\n")

# Get the indexes of patches that are within the polygons
tumour_patch_indexes = sorted(get_patches_within_polygons(patch_centers, polygons))
print(f"{len(tumour_patch_indexes)} Tumour patch indexes:\n", tumour_patch_indexes[:5], "...\n")

In [ ]:
print("In tumour_patch_indexes (computed now), but not in gt_patches_indexes_np (shared by the authors):", len(set(tumour_patch_indexes) - set(gt_patches_indexes_np)))
print("In gt_patches_indexes_np (shared by the authors), but not in tumour_patch_indexes (computed now):", len(set(gt_patches_indexes_np) - set(tumour_patch_indexes)))

## ASAP for making a mask from xml annotation files

1. Download [ASAP](https://github.com/computationalpathologygroup/ASAP)

2. Add these lines to `~/.bashrc`:
    ```
    # Add the path to the ASAP (https://github.com/computationalpathologygroup/ASAP) bin directory
    export PYTHONPATH=$PYTHONPATH:/opt/ASAP/bin
    ```
   
3. Close and open shell for the changes to take place.

Instructions from: https://camelyon17.grand-challenge.org/Data/


**Issues that it's useful to check**

1. **Problem**: Mask empty: https://github.com/computationalpathologygroup/ASAP/issues/205
**Solution**: Fix label map to correspond to the xml groups. See issue comment.


2. **Problem**: Opening mask with openslide results in an empty mask, while ASAP works: https://github.com/computationalpathologygroup/ASAP/issues/143
**Solution**: Be careful with the channels when opening openslide. See issue comment.


3. **Problem**: Mask making is too slow: https://github.com/computationalpathologygroup/ASAP/issues/195
**Solution**: Lower the resolution, **how???**

In [ ]:
reader = mir.MultiResolutionImageReader()
mr_image = reader.open(sample_slide_path)

image_thumbnail = mr_image.getUCharPatch(
    0, 0,
    mr_image.getLevelDimensions(mask_downsapmle_level)[0], mr_image.getLevelDimensions(mask_downsapmle_level)[1],
    mask_downsapmle_level
)
plt.imshow(image_thumbnail)

In [ ]:
downsample_factor = mr_image.getLevelDownsample(example_region_downsample_level)
print(downsample_factor)
image_patch = mr_image.getUCharPatch(
    *example_region_xy,     # *(x, y) -> x, y
    *example_region_size,
    example_region_downsample_level
)
plt.imshow(image_patch)
image_patch.size

In [ ]:
annotation_list = mir.AnnotationList()
xml_repository = mir.XmlRepository(annotation_list)
xml_repository.setSource(sample_xml_annotation_path)
xml_repository.load()

In [ ]:
[el for el in dir(annotation_list.getAnnotation(0)) if not el.startswith('_')]

In [ ]:
print(annotation_list.getAnnotation(0).getName())
print(annotation_list.getAnnotation(0).getNumberOfPoints())


In [ ]:
annotation_mask = mir.AnnotationToMask()

dataset_2_convert_args = {
    'camelyon16': {
        'label_map': {'Tumor': 1, 'Normal': 2},
        'conversion_order': ['Tumor', 'Normal']
    },
    'camelyon17': {
        'label_map': {'metastases': 1, 'normal': 2},
        'conversion_order': ['metastases', 'normal']
    },
    'default': {
        'label_map': {'_0': 1, '_1': 1, '_2': 0},
        'conversion_order': ['_0', '_1', '_2']
    }
}

dataset_name = 'camelyon16'
label_map = dataset_2_convert_args.get(dataset_name, dataset_2_convert_args['default'])['label_map']
conversion_order = dataset_2_convert_args.get(dataset_name, dataset_2_convert_args['default'])['conversion_order']

print(label_map)
print(conversion_order)


In [ ]:
annotation_mask.convert(
    annotation_list,
    sample_from_xml_reference_mask_mir_path,
    mr_image.getDimensions(),
    mr_image.getSpacing(),
    label_map,
    conversion_order,
)

In [ ]:
print("sample_from_xml_reference_mask_mir_path", sample_from_xml_reference_mask_mir_path)
print("mask_downsapmle_level", mask_downsapmle_level)

mr_mask = reader.open(sample_from_xml_reference_mask_mir_path)
print("Mask Dimentions", mr_mask.getDimensions())
print("Min value:", mr_mask.getMinValue())
print("Max value:", mr_mask.getMaxValue())


mr_mask_thumbnail = mr_mask.getUCharPatch(
    0, 0,
    mr_image.getLevelDimensions(mask_downsapmle_level)[0], mr_image.getLevelDimensions(mask_downsapmle_level)[1],
    mask_downsapmle_level
)
plt.imshow(mr_mask_thumbnail, cmap='gray')

In [ ]:
print("sample_from_xml_reference_mask_mir_path", sample_from_xml_reference_mask_mir_path)
print("mask_downsapmle_level", mask_downsapmle_level)

openslide_mask = openslide.open_slide(sample_from_xml_reference_mask_mir_path)

openslide_mask_thumbnail = openslide_mask.read_region(
    (0, 0),
    mask_downsapmle_level,
    openslide_mask.level_dimensions[mask_downsapmle_level]
)
openslide_mask_thumbnail_np = np.array(openslide_mask_thumbnail)

print("Shape:", openslide_mask_thumbnail_np.shape)
print("Min values across channels:", openslide_mask_thumbnail_np.min(axis=(0,1)))
print("Max values across channels", openslide_mask_thumbnail_np.max(axis=(0,1)))
plt.imshow(openslide_mask_thumbnail_np[:, :, 0], cmap='gray')

In [ ]:
openslide_spacing = (
    float(openslide_slide.properties['openslide.mpp-x']),
    float(openslide_slide.properties['openslide.mpp-y'])
    )

# possible to use openslide_slide.dimensions and openslide_spacing
# annotation_mask.convert(
#     annotation_list,
#     sample_from_xml_reference_mask_openslide_path,
#     openslide_slide.dimensions,
#     openslide_spacing,
#     label_map,
#     conversion_order
# )

In [ ]:
assert mr_image.getDimensions() == openslide_slide.dimensions
assert np.allclose(np.array(mr_image.getSpacing()), np.array(openslide_spacing))